# ☁️ Week 11-12 · Notebook 06 · Deploying to Google Cloud Platform (GCP)

This notebook provides a practical guide to deploying our containerized Manufacturing Copilot application to Google Cloud, focusing on **Cloud Run** for serving and **Cloud Functions** for event-driven processing.


## 🎯 Learning Objectives

- **Deploy to Cloud Run:** Configure and deploy our Docker container to Cloud Run, a serverless platform, enabling automatic scaling, version management, and HTTPS endpoints.
- **Manage Secrets Securely:** Use Google Secret Manager to store and securely inject sensitive data (like API keys and database passwords) into the running container.
- **Automate with Deployment Scripts:** Write Python scripts that use the `gcloud` CLI to automate and standardize the deployment process across different environments (staging, production).
- **Implement Event-Driven Error Notifications:** Set up a pipeline using Cloud Logging, Pub/Sub, and Cloud Functions to automatically send notifications to a Teams channel or PagerDuty when errors occur.


## 🧩 Scenario: A Scalable, Multi-Region Copilot Service

The Manufacturing Copilot is a global success, and it needs to be deployed in multiple regions to serve plants in India (`asia-south1`) and Mexico (`us-central1`) with low latency.

**The Architecture Requirements:**
1.  **Primary Service:** The main FastAPI application will be deployed on **Cloud Run**. It must scale automatically based on traffic, from zero to a configured maximum.
2.  **Asynchronous Processing:** User feedback (e.g., ratings on a RAG response) will be handled by a separate, lightweight **Cloud Function** that ingests the feedback and stores it in BigQuery without blocking the main application.
3.  **Secure Configuration:** All prompts will be stored centrally in a Cloud Storage bucket, and all secrets (API keys, etc.) will be managed by Google Secret Manager.
4.  **Private Networking:** The Cloud Run service needs to access an on-premises database, so it must be connected to a Virtual Private Cloud (VPC) that is linked to the factory network via Cloud VPN.


## 🗺️ GCP Deployment Architecture

This diagram illustrates how the different GCP services work together to host the Manufacturing Copilot.

```
[Technician's Tablet] --HTTPS--> [Global Load Balancer]
                                       |
                                       +--> [Cloud Run: copilot-api-prod (us-central1)]
                                       |
                                       +--> [Cloud Run: copilot-api-prod (asia-south1)]

[Cloud Run Service]
 |
 +-- Reads prompts from [Cloud Storage Bucket]
 |
 +-- Accesses secrets from [Secret Manager]
 |
 +-- Connects to on-prem DB via [VPC Connector] -> [Cloud VPN]
 |
 +-- Calls [Vertex AI API] for embeddings/models
 |
 +-- Sends logs to [Cloud Logging]

[User Feedback] --HTTPS--> [Cloud Function: feedback-ingestor] --> [BigQuery]

[Cloud Logging] --Log Sink--> [Pub/Sub Topic: error-notifications] --> [Cloud Function: incident-notifier] --> [Teams/PagerDuty]
```


In [ ]:
# --- scripts/deploy_cloud_run.py ---
# A Python script to standardize Cloud Run deployments.

import subprocess
import shlex
from dataclasses import dataclass, field
from typing import Dict

@dataclass
class CloudRunConfig:
    """Configuration for a Cloud Run deployment."""
    service_name: str
    image_uri: str
    region: str
    project_id: str
    max_instances: int = 4
    min_instances: int = 0  # Set to 1 for no cold starts, 0 for cost savings
    vpc_connector: str = "" # Optional VPC connector name
    secrets: Dict[str, str] = field(default_factory=dict) # Secret name -> Env var name

def deploy_to_cloud_run(config: CloudRunConfig):
    """Constructs and runs the gcloud command to deploy a service."""
    
    base_command = f"""
        gcloud run deploy {config.service_name} \\
            --image {config.image_uri} \\
            --region {config.region} \\
            --project {config.project_id} \\
            --platform managed \\
            --allow-unauthenticated \\
            --min-instances {config.min_instances} \\
            --max-instances {config.max_instances}
    """
    
    if config.vpc_connector:
        base_command += f" --vpc-connector {config.vpc_connector}"
        
    if config.secrets:
        secret_args = [f"{env_var}={name}:latest" for name, env_var in config.secrets.items()]
        base_command += f" --update-secrets={','.join(secret_args)}"

    # Use shlex to safely parse the command string
    command_list = shlex.split(base_command.strip())
    
    print("--- Running Deployment Command ---")
    print(" ".join(command_list))
    
    try:
        # In a real script, you would run this command.
        # subprocess.run(command_list, check=True, capture_output=True, text=True)
        print("\n--- Deployment Successful (Simulated) ---")
    except subprocess.CalledProcessError as e:
        print(f"\n--- Deployment Failed ---")
        print(f"Error: {e.stderr}")
        
if __name__ == "__main__":
    # Example usage for deploying to the staging environment
    staging_config = CloudRunConfig(
        project_id="manufacturing-copilot-dev",
        service_name="copilot-api-staging",
        image_uri="gcr.io/manufacturing-copilot-dev/copilot-api:sha-123abc",
        region="us-central1",
        min_instances=0,
        max_instances=2,
        secrets={
            "OPENAI_API_KEY": "ENV_OPENAI_API_KEY",
            "DB_PASSWORD_STAGING": "ENV_DB_PASSWORD"
        }
    )
    deploy_to_cloud_run(staging_config)


In [ ]:
# --- scripts/deploy_cloud_function.py ---
# A shell script is often simpler for deploying a single Cloud Function.

deploy_function_script = """
#!/bin/bash

# Deploys the feedback ingestor Cloud Function

PROJECT_ID="manufacturing-copilot-dev"
REGION="us-central1"
FUNCTION_NAME="feedback-ingestor"
ENTRY_POINT="ingest_feedback"
RUNTIME="python310"
TRIGGER="--trigger-http" # HTTP-triggered function
SOURCE_DIR="./src/functions/feedback"
BIGQUERY_TABLE="feedback.user_ratings"

echo "Deploying function ${FUNCTION_NAME} to project ${PROJECT_ID}..."

gcloud functions deploy ${FUNCTION_NAME} \\
  --project=${PROJECT_ID} \\
  --region=${REGION} \\
  --runtime=${RUNTIME} \\
  --entry-point=${ENTRY_POINT} \\
  ${TRIGGER} \\
  --source=${SOURCE_DIR} \\
  --allow-unauthenticated \\
  --set-env-vars=BIGQUERY_TABLE=${BIGQUERY_TABLE}

echo "Deployment complete."
"""

print("--- Cloud Function Deployment Script ---")
print(deploy_function_script)
# In a real project, you would save this as a .sh file and execute it.


## 🔐 Securely Managing Secrets and Configuration

Hardcoding secrets is a major security risk. We use specific GCP services to manage different types of configuration.

| Configuration Type      | GCP Service             | How it's Used                                                                                             |
| ----------------------- | ----------------------- | --------------------------------------------------------------------------------------------------------- |
| **Sensitive Secrets**   | **Secret Manager**      | API keys, database passwords, signing keys. Injected into Cloud Run as environment variables or mounted as files. |
| **Non-Sensitive Config**| **Environment Variables** | `LOG_LEVEL`, `PLANT_ID`. Set directly on the Cloud Run service during deployment.                         |
| **Large Config Files**  | **Cloud Storage**       | LLM prompt templates, model configuration YAMLs. The application fetches these from a GCS bucket on startup. |

### Example: Accessing a Secret in Python

```python
# This code would run inside the Cloud Run container.
import os

# The secret was mapped to an environment variable during deployment.
api_key = os.getenv("ENV_OPENAI_API_KEY")

if not api_key:
    raise ValueError("OpenAI API key not found in environment variables.")

# Now you can use the api_key to interact with the OpenAI client.
```


## ⚠️ Automated Error Notification Pipeline

We can build a powerful, serverless pipeline to automatically notify us of problems.

1.  **Log Sink:**
    -   Go to the GCP Console > Logging > Log Router.
    -   Create a sink with a filter like `severity>=ERROR AND resource.type="cloud_run_revision"`.
    -   Set the destination to a new Pub/Sub topic called `error-notifications`.

2.  **Notifier Cloud Function:**
    -   Create a new Cloud Function (`incident-notifier`) that is triggered by messages on the `error-notifications` Pub/Sub topic.
    -   This function's code will parse the log entry and format a message to be sent to a webhook.

    ```python
    # main.py for the incident-notifier Cloud Function
    import base64
    import json
    import requests

    TEAMS_WEBHOOK_URL = "https://your-webhook-url.office.com/..."

    def send_error_notification(event, context):
        """Triggered by a message on a Pub/Sub topic."""
        log_entry = json.loads(base64.b64decode(event['data']).decode('utf-8'))
        
        service_name = log_entry['resource']['labels']['service_name']
        error_text = log_entry['textPayload']
        trace_id = log_entry['trace'].split('/')[-1]

        message = {
            "@type": "MessageCard",
            "summary": f"Critical Error in {service_name}",
            "text": f"**Service:** {service_name}\\n\\n**Error:** {error_text}\\n\\n**Trace ID:** {trace_id}"
        }
        
        requests.post(TEAMS_WEBHOOK_URL, json=message)
    ```

3.  **Result:**
    -   Now, every time your Cloud Run service logs an error, a message will automatically appear in your configured Microsoft Teams channel within seconds.


## 🧪 Lab Assignment: Deploy the Copilot

1.  **Set Up Your GCP Project:**
    -   Create a new GCP project.
    -   Enable the required APIs: Cloud Run, Artifact Registry, Secret Manager, Cloud Build.
    -   Create a service account for GitHub Actions to use for deployment.

2.  **Push Your Container Image:**
    -   Configure Docker to authenticate with Artifact Registry: `gcloud auth configure-docker`.
    -   Build your Docker image and tag it with the Artifact Registry path: `docker build -t us-central1-docker.pkg.dev/your-project/copilot-repo/copilot-api:latest .`
    -   Push the image: `docker push us-central1-docker.pkg.dev/your-project/copilot-repo/copilot-api:latest`.

3.  **Create Secrets:**
    -   In Secret Manager, create a secret for a placeholder API key.

4.  **Deploy to Cloud Run:**
    -   Modify and run the `deploy_cloud_run.py` script from this notebook, pointing it to your project, service name, and image URI.
    -   Make sure to map the secret you created to an environment variable.
    -   Once deployed, access the public URL provided by Cloud Run to test your service.

5.  **Set Up a Health Check:**
    -   Create a Cloud Scheduler job that sends a GET request to your service's `/health` endpoint every 5 minutes.
    -   Configure it to post to a Pub/Sub topic if the health check fails.


## ✅ Checklist for this Notebook

- [X] Deployment architecture designed for multi-region, serverless deployment on GCP.
- [X] Automated deployment scripts created for both Cloud Run and Cloud Functions.
- [X] Secure configuration strategy defined using Secret Manager and Cloud Storage.
- [X] A serverless error notification pipeline designed to provide real-time alerts.
- [ ] **TODO:** Complete the Lab Assignment to deploy the containerized application to your own GCP project.


## 📚 References and Further Reading

-   [GCP Cloud Run Documentation](https://cloud.google.com/run/docs)
-   [GCP Secret Manager Documentation](https://cloud.google.com/secret-manager/docs)
-   [Deploying to Cloud Run from GitHub Actions](https://github.com/google-github-actions/deploy-cloudrun)
-   [Cloud Logging Sinks](https://cloud.google.com/logging/docs/export/configure_export_v2)
-   [Tutorial: Serverless CI/CD on Google Cloud](https://cloud.google.com/architecture/serverless-ci-cd)
